In [1]:
from lalsimulation.unit import LAL_G_C4_SI

ImportError: cannot import name 'LAL_G_C4_SI' from 'lalsimulation' (/cvmfs/oasis.opensciencegrid.org/ligo/sw/conda/envs/igwn-py39/lib/python3.9/site-packages/lalsimulation/__init__.py)

In [ ]:
epsabs = 0.0
epsrel = 1e-6
def TOV_integrate(central_pressure_si, eos):
    """
    /* central values */
    /* note: will be updated with Lindblom's series expansion */
    /* geometrisized units for variables in length (m) */
     """
    pc = central_pressure_si * LAL_G_C4_SI;
    ec = XLALSimNeutronStarEOSEnergyDensityOfPressureGeometerized(pc, eos);
 double hc =
     XLALSimNeutronStarEOSPseudoEnthalpyOfPressureGeometerized(pc, eos);
 double dedp_c =
     XLALSimNeutronStarEOSEnergyDensityDerivOfPressureGeometerized(pc, eos);
 double dhdp_c = 1.0 / (ec + pc);
 double dedh_c = dedp_c / dhdp_c;
 double dh = -1e-3 * hc;
 double h0 = hc + dh;
 double h1 = 0.0 - dh;
 double r0 = sqrt(-3.0 * dh / (2.0 * LAL_PI * (ec + 3.0 * pc)));
 double m0 = 4.0 * LAL_PI * r0 * r0 * r0 * ec / 3.0;
 double H0 = r0 * r0;
 double b0 = 2.0 * r0;

 double yy;
 double c;
 double h;
 size_t i;

 /* series expansion for the initial core */

 /* second factor of Eq. (7) of Lindblom (1992) */
 r0 *= 1.0 + 0.25 * dh * (ec - 3.0 * pc  - 0.6 * dedh_c) / (ec + 3.0 * pc);
 /* second factor of Eq. (8) of Lindblom (1992) */
 m0 *= 1.0 + 0.6 * dh * dedh_c / ec;

 /* perform integration */
 vars->r = r0;
 vars->m = m0;
 vars->H = H0;
 vars->b = b0;
 h = h0;
 while (h > h1) {
     int s =
         gsl_odeiv_evolve_apply(evolv, ctrl, step, &sys, &h, h1, &dh, y);
     if (s != GSL_SUCCESS)
         XLAL_ERROR(XLAL_EERR,
             "Error encountered in GSL's ODE integrator\n");
 }

 /* take one final Euler step to get to surface */
 tov_ode(h, y, dy, eos);
 for (i = 0; i < TOV_ODE_VARS_DIM; ++i)
     y[i] += dy[i] * (0.0 - h1);

 /* compute tidal Love number k2 */
 c = vars->m / vars->r;      /* compactness */
 yy = vars->r * vars->b / vars->H;

 /* convert from geometric units to SI units */
 *radius = vars->r;
 *mass = vars->m * LAL_MSUN_SI / LAL_MRSUN_SI;
 *love_number_k2 = tidal_Love_number_k2(c, yy);

 /* free ode memory */
 gsl_odeiv_evolve_free(evolv);
 gsl_odeiv_control_free(ctrl);
 gsl_odeiv_step_free(step);
 return 0;